In [2]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from config import user, password, db_name

### Extract

In [3]:
# Import CSV with AirBnB data
csv_file = "Resources/listings.csv"
airbnb_data_df = pd.read_csv(csv_file)
airbnb_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,60,2,178,2019-12-15,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,2020-07-14,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,2020-03-08,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,2019-10-23,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2020-02-14,0.61,5,168


In [5]:
airbnb_data_df = airbnb_data_df.drop(["neighbourhood_group"], axis=1)

In [7]:
airbnb_data_df.set_index("id", inplace=True)

In [19]:
airbnb_data_df.dropna(how = "any", inplace=True)
airbnb_data_df.head()

,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,
2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,60,2,178,2019-12-15,2.56,1,353
4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,2020-07-14,2.81,1,155
7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,2020-03-08,2.81,1,321
9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,2019-10-23,0.63,9,300
10610,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2020-02-14,0.61,5,168


In [98]:
#listing_columns = ["host_id", "host_name", "minimum nights"]
listing_info_df = airbnb_data_df.iloc[:, [1, 2, 6, 7, 8, 9, 10, 11, 12, 13]]

#df.columns
#df.iloc[:, 2]
location_info_df = airbnb_data_df.iloc[:, [0, 3, 4, 5]]

In [99]:
location_info_df = location_info_df.rename(columns={"name": "address"})

In [100]:
listing_info_df
location_info_df

,address,neighbourhood,latitude,longitude
id,,,,
2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",Hyde Park,41.78790,-87.58780
4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,South Lawndale,41.85495,-87.69696
7126,Tiny Studio Apartment 94 Walk Score,West Town,41.90289,-87.68182
9811,Barbara's Hideaway - Old Town,Lincoln Park,41.91769,-87.63788
10610,3 Comforts of Cooperative Living,Hyde Park,41.79612,-87.59261
...,...,...,...,...
45351578,Spacious Modern Vibe Avondale/Logan -Entire Apt,Avondale,41.94060,-87.72281
45368527,Private Bedroom in Chicago Near Lincoln Square...,North Park,41.99222,-87.71659
45386114,BEST PRICE for a studio in Wrigleyville-Boystown,Uptown,41.95513,-87.65095


In [35]:
# Import CSV with community area numbers
csv_file = "Resources/chicago-community-areas.csv"
communities_df = pd.read_csv(csv_file)
communities_df.head()

,Community Area,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,name,Rogers Park,West Ridge,Uptown,Lincoln Square,North Center,Lake View,Lincoln Park,Near North Side,Edison Park,...,Englewood,Greater Grand Crossing,Ashburn,Auburn Gresham,Beverly,Washington Heights,Mount Greenwood,Morgan Park,O'Hare,Edgewater


In [37]:
communities_df.columns = communities_df.columns.str.lower()

# Replace spaces in column names with underscores
communities_df.columns = communities_df.columns.str.replace(' ', '_')

communities_df.set_index("community_area", inplace=True)

In [38]:
communities_df = communities_df.transpose()

In [41]:
communities_df = communities_df.rename(columns={"name": "neighborhood"})

In [42]:
communities_df

community_area,neighborhood
1,Rogers Park
2,West Ridge
3,Uptown
4,Lincoln Square
5,North Center
...,...
73,Washington Heights
74,Mount Greenwood
75,Morgan Park
76,O'Hare


In [60]:
# Import CSV with Chicago 2013 population numbers
csv_file = "Resources/chicago_population2013.csv"
population_df = pd.read_csv(csv_file, sep=';')

In [61]:
# Make the column names lowercase
population_df.columns = population_df.columns.str.lower()

# Replace spaces in column names with underscores
population_df.columns = population_df.columns.str.replace(' ', '_')

In [63]:
population_df.set_index("community_area", inplace=True)
population_df = population_df.transpose()

In [64]:
population_df.head()

community_area,name,population,income,latinos,blacks,white,asian,other
1,Rogers Park,54991,39482,0.244,0.263,0.393,0.064,0.036
2,West Ridge,71942,47323,0.204,0.111,0.427,0.225,0.032
3,Uptown,56362,40324,0.142,0.2,0.516,0.114,0.028
4,Lincoln Square,39493,57749,0.191,0.038,0.631,0.111,0.029
5,North Center,31867,81524,0.136,0.023,0.773,0.045,0.022


### Transform

In [ ]:
# This code does not run by itself: this is sample code

# Make the column names lowercase
df.columns = df.columns.str.lower()

# Replace spaces in column names with underscores
df.columns = df.columns.str.replace(' ', '_')

# Rename Columns
df = df.rename(columns={"OLD NAME": "new_name", "OLD NAME2": "new_name_2"})

# Drop duplicates in column before turning it into the index
df.drop_duplicates("column", inplace=True)
df.set_index("column", inplace=True)

# Choose columns to retain
preferred_columns = ["keep", "these", "columns"]
df_transformed = df[preferred_columns].copy()




### Load

In [11]:
# Create database connection
rds_connection_string = f"{user}:{password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
# Check for tables
inspector = inspect(engine)
print(inspector.get_table_names())

[]


In [ ]:
# Load CSV converted DataFrame into database
df.to_sql(name='table_name', con=engine, if_exists='append', index=False)

In [ ]:
# Confirm data has been added
pd.read_sql_query('select * from table_name', con=engine).head()